In [ ]:
from langchain_core.tools import tool
from dotenv import load_dotenv
import sys,os
from io import StringIO
from pydantic import BaseModel, Field
import pandas as pd
from typing import Optional


In [28]:
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle # <-- ADD THIS IMPORT
from reportlab.platypus import Paragraph, Frame # <-- ADD FRAME IMPORT


In [29]:
@tool
def eda_fact_sheet(path:str,sample_size: int = 3, freq_top: int = 3):
    """
    Generates a batchwise fact sheet for a dataset.

    Args:
        path (str): Path to CSV or Excel dataset.
        sample_size (int): Number of example values per column.
        freq_top (int): Number of top frequent values per column.

    Returns:
        dict: Fact sheet containing dataset-level info and column-level stats in batches.
    """
    path = path.strip().strip('"').strip("'")
    path = os.path.normpath(path)

    #--- Load dataset ---
    if path.endswith(".csv"):
        df = pd.read_csv(path)
    elif path.endswith(".xlsx") or path.endswith(".xls"):
        df = pd.read_excel(path)
    else:
        raise ValueError("Unsupported file format. Use CSV or Excel.")

    df = pd.read_csv(path)

    n_rows, n_columns = df.shape

    fact_sheet = {
        "n_rows": n_rows,
        "n_columns": n_columns,
        "batches": []
    }

    # --- Determine batch splits ---
    batch_size = 15
    batch_splits = [df.columns[i:i+batch_size].tolist() for i in range(0, n_columns, batch_size)]

    # --- Process each batch ---
    for batch_cols in batch_splits:
        batch_profile = {"columns": {}}
        for col in batch_cols:
            series = df[col]
            col_profile = {}

            total = len(series)

            # Core stats
            col_profile["dtype"] = str(series.dtype)
            col_profile["null_percent"] = round(float(series.isna().sum() / total * 100), 2)
            col_profile["unique_percent"] = round(float(series.nunique(dropna=True) / total * 100), 2)

            # Example values
            try:
                col_profile["examples"] = series.dropna().sample(
                    min(sample_size, series.dropna().shape[0]),
                    random_state=42
                ).tolist()
            except ValueError:
                col_profile["examples"] = []

            # Top frequent values
            if not series.isna().all():
                top_freq = series.value_counts(dropna=True).head(freq_top)
                col_profile["top_values"] = {str(k): int(v) for k, v in top_freq.to_dict().items()}

            # Numeric columns
            if pd.api.types.is_numeric_dtype(series):
                col_profile.update({
                    "min": float(series.min(skipna=True)),
                    "max": float(series.max(skipna=True)),
                    "mean": float(series.mean(skipna=True)),
                    "std": float(series.std(skipna=True))
                })
                if series.std(skipna=True) > 0:
                    z_scores = ((series - series.mean(skipna=True)) / series.std(skipna=True)).abs()
                    col_profile["has_outliers"] = bool((z_scores > 3).any())
                else:
                    col_profile["has_outliers"] = False

            # Datetime columns
            elif pd.api.types.is_datetime64_any_dtype(series):
                if not series.dropna().empty:
                    col_profile["min_date"] = str(series.min())
                    col_profile["max_date"] = str(series.max())

            # Text/categorical columns
            elif pd.api.types.is_object_dtype(series):
                lengths = series.dropna().astype(str).map(len)
                if not lengths.empty:
                    col_profile["avg_length"] = float(lengths.mean())
                    col_profile["max_length"] = int(lengths.max())
                unusual = series.dropna().astype(str).str.contains(r"[^a-zA-Z0-9\s]").sum()
                col_profile["unusual_char_percent"] = round(float(unusual / total * 100), 2)

            # Flags
            if series.nunique(dropna=True) == total:
                col_profile["is_identifier"] = True
            elif series.nunique(dropna=True) <= 1:
                col_profile["is_constant"] = True

            batch_profile["columns"][col] = col_profile

        fact_sheet["batches"].append(batch_profile)

    return fact_sheet


In [30]:
class PythonInputs(BaseModel):
    query: str = Field(description="A valid python command to run.")

load_dotenv()

@tool(args_schema=PythonInputs)
def python_repl_ast(query: str, path: Optional[str] = None) -> str:
    """
    Runs a Python command and returns the result.
    If `path` is provided, the dataframe at that path will be loaded as `df`.
    Supports CSV, Excel, and Parquet. 
    Any in-place modifications to `df` are saved back to the same file.
    """

    df = None
    if path:
        ext = os.path.splitext(path)[-1].lower()
        if ext == ".csv":
            df = pd.read_csv(path)
        elif ext in [".xls", ".xlsx"]:
            df = pd.read_excel(path)
        elif ext == ".parquet":
            df = pd.read_parquet(path)
        else:
            return f"Unsupported file type: {ext}"

    # Restricted namespaces
    local_namespace = {"df": df}
    global_namespace = {}

    # Capture stdout for print()
    old_stdout = sys.stdout
    sys.stdout = captured_output = StringIO()

    try:
        # Run code
        exec(query, global_namespace, local_namespace)

        # Capture output
        output = captured_output.getvalue()
        if not output:
            try:
                # Eval last expression if no print output
                lines = [line for line in query.strip().split("\n") if line.strip()]
                if lines:
                    last_line_result = eval(lines[-1], global_namespace, local_namespace)
                    output = str(last_line_result)
                else:
                    output = "Executed successfully, but no output was produced."
            except Exception:
                output = "Executed successfully, but no output was produced."

        # Save back modified DataFrame if applicable
        if path and "df" in local_namespace and isinstance(local_namespace["df"], pd.DataFrame):
            df = local_namespace["df"]
            if ext == ".csv":
                df.to_csv(path, index=False)
            elif ext in [".xls", ".xlsx"]:
                df.to_excel(path, index=False)
            elif ext == ".parquet":
                df.to_parquet(path, index=False)

        return output

    except Exception as e:
        return f"Execution failed with error: {e}"

    finally:
        sys.stdout = old_stdout


In [31]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

# Load environment variables
load_dotenv()

# Get API key from .env
api_key = os.getenv("GOOGLE_API_KEY")

if not api_key:
    raise ValueError("GOOGLE_API_KEY not found. Make sure it's set in the .env file.")

# Initialize Gemini
llm_model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=api_key
)


In [32]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
system_prompt = ChatPromptTemplate.from_messages(
    [
        (
           "system",
            "You are a Business report agent. You have access to a tool `eda_fact_sheet(df_path)` which generates a full fact sheet for a CSV/DataFrame. "
            "The csv is in this path : {df_path}. Use the tool first, then provide a comprehensive business report based on the dataset analysis."
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad")        
    ]
)

In [33]:
from langchain.agents import AgentExecutor, create_tool_calling_agent

In [34]:
Analyzer_agent = create_tool_calling_agent(
            llm=llm_model,
            tools=[eda_fact_sheet,python_repl_ast],
            prompt=system_prompt
        )

tools = [eda_fact_sheet,python_repl_ast]

In [35]:
path = r"D:\Code Assistant\avocado.csv"

In [36]:
Instructions = "Data is about sales,provide the data overview along with the preprocessing steps needed to perform EDA , here's the path = {path}"

In [37]:
Reporter_prompt = """You are an elite business intelligence consultant, renowned for your ability to parachute into any business, analyze a raw dataset, and emerge with a strategic plan that creates significant value. Your client, a business owner, has given you a CSV file and a simple request: "Find the single biggest opportunity hidden in this data and tell me exactly how to capitalize on it."

Your mission: Perform a comprehensive, hypothesis-driven analysis of the provided data. You must autonomously identify the key metrics, dimensions, and relationships within the dataset to build a compelling business case study. Your analysis must culminate in a detailed report that explores the "what," the "why," and the "how" to drive business growth.

Your Tools:
1. `eda_fact_sheet(df_path)`: Use this once at the very beginning to perform initial data reconnaissance and understand the landscape of the dataset (columns, data types, etc.).
2. `python_repl_ast`: This is your primary analytical tool. You will use it multiple times to explore hypotheses, segment the data, and quantify your findings.

---

### Your Mandated Investigative Framework:

You are not being given specific questions. It is your job as an expert to formulate and answer them. Your investigation must follow this strategic framework:

1.  **Data Reconnaissance & Profiling:** Use `eda_fact_sheet` to get a high-level overview. What are the columns? What are the data types? Are there obvious missing values?

2.  **Hypothesis Generation & Deep-Dive Analysis:** After your initial recon, you must form hypotheses about the data and use `python_repl_ast` to test them. Your deep dive should be structured to uncover:
    - **Core Metric Identification:** What are the most important performance indicators (KPIs) in this dataset? (e.g., `Sales`, `Revenue`, `Profit`, `User_Activity`, `Conversion_Rate`). You must decide which columns represent the core business goals.
    - **Dimensional Analysis:** How do the core metrics perform across different categories or segments? You must identify the key dimensional columns (e.g., `Region`, `Product_Category`, `Customer_Segment`, `Store_Type`) and use them to slice and dice the data to find top and bottom performers.
    - **Temporal Analysis:** Are there trends over time? Look for date/time columns and analyze for growth, decline, seasonality, or other time-based patterns (e.g., year-over-year growth, peak hours, quarterly trends).
    - **Correlation & Relationship Analysis:** How do different numerical features interact? For example, does `Marketing_Spend` correlate with `Sales`? Does `Discount_Rate` affect `Profit_Margin`?
    - **Segmentation & Outlier Analysis:** Can you identify distinct groups or clusters in the data? Are there significant outliers that represent either a major problem or a unique opportunity?

3.  **Synthesize and Report:** After your investigation, you must weave all your quantitative findings into the mandatory report format below. Your narrative must connect the dots between your different findings to tell a single, coherent strategic story.

---

### Mandatory Report Format: Business Case Study & Strategic Plan

**Subject: Strategic Deep-Dive: Unlocking Growth Opportunities in [Dataset Theme - *You determine this*]**

**(1) Executive Summary:**
*A brief, 3-4 sentence overview for a C-suite audience. State the overall condition of the business based on the data, the single most significant opportunity you have uncovered, and the high-level recommendation.*

**(2) Current State of the Business: A Data-Driven Overview:**
*Provide a clear overview of the macro trends, supported by the key metrics you identified.*
-   Present the overall trend of the 1-2 primary KPIs you identified (e.g., "Overall sales have shown a 15% compound annual growth rate from 2020-2023").
-   Briefly describe the key segments or dimensions you discovered in the data (e.g., "The business operates across four primary regions and sells three distinct product categories").

**(3) The Core Insight: Identifying the Single Biggest Opportunity:**
*This is the heart of your case study. Detail your main finding with specific, hard numbers from your analysis.*
-   Clearly identify the top-performing segment that represents the biggest opportunity (e.g., a specific product line, customer demographic, or geographic region).
-   Quantify the performance gap. State the exact difference in performance between the top and bottom segments (e.g., "The 'Enterprise' customer segment generates a 300% higher lifetime value than the 'SMB' segment").
-   Expose a potential Resource Allocation Mismatch. Based on the data, hypothesize if the company's resources (e.g., supply, marketing spend, personnel) are appropriately focused on this high-value opportunity.

**(4) Supporting Analysis: Deeper Insights & Corroborating Evidence:**
*Add layers to your story with other detailed findings that support your core insight.*
-   **Key Temporal Patterns:** Present any seasonal trends, growth patterns, or time-of-day effects that are relevant to the core opportunity (e.g., "The high-value 'Enterprise' segment makes 60% of its purchases in the final quarter of the fiscal year").
-   **Behavioral Analysis:** Describe any patterns in behavior associated with the core opportunity (e.g., "These top customers overwhelmingly purchase 'Service Plan A' but rarely buy 'Service Plan B', suggesting an opportunity for cross-selling").

**(5) The Go-Forward Strategic Plan: A Phased Approach**
*Provide a detailed, multi-step action plan based on your findings.*

**Phase 1: Immediate Realignment & Proof of Concept (Next 30 Days)**
1.  **Strategic Focus Shift:** Detail an immediate action to pivot towards the identified opportunity (e.g., "Reroute 20% of the digital marketing budget from the underperforming 'SMB' campaigns to a targeted pilot campaign for the 'Enterprise' segment").
2.  **Initial Test & Measurement:** Specify a small-scale test to validate your hypothesis (e.g., "Offer a 10% discount on 'Service Plan B' to a test group of 100 'Enterprise' customers to measure uplift in adoption").

**Phase 2: Scale & Market Capture (Next Quarter)**
1.  **Full-Scale Rollout:** Based on the results of Phase 1, describe a broader initiative (e.g., "Launch a company-wide marketing and sales initiative focused on the 'Enterprise' segment, timed to align with the peak Q4 purchasing window").
2.  **Operational Adjustments:** Recommend changes to business operations based on your behavioral analysis (e.g., "Adjust inventory or staffing levels to prepare for the predictable Q4 demand surge").

**Metrics for Success:**
-   We will measure the success of this plan by tracking the following KPIs:
    -   Increase in the primary KPI (e.g., revenue, profit) from the target segment.
    -   Improvement in a key efficiency metric (e.g., Customer Acquisition Cost, Return on Ad Spend).
    -   Change in a key behavioral metric (e.g., cross-sell adoption rate, customer churn).

---

### **CRITICAL DIRECTIVES:**
- **AUTONOMOUS ANALYSIS IS KEY:** You are not given a list of questions. It is your job as an expert consultant to analyze the dataset, determine the most important business questions, and answer them.
- **DEPTH IS MANDATORY:** A shallow report is a failed report. Your conclusions must be supported by specific numbers, percentages, and trends you discover using your tools.
- **CONNECT THE DOTS:** Do not just list disconnected facts. Your report must tell a compelling and coherent story, where each piece of analysis builds on the last to support your final recommendation.
- **STRATEGIC, NOT JUST ACTIONABLE:** Your plan must have logical phases and clear metrics for success, demonstrating a thoughtful, long-term approach to creating business value."""

In [38]:
# Reporter_prompt = """You are a top-tier business strategy consultant. Your client, the business owner, has tasked you with a deep-dive analysis of their sales data. They need a comprehensive business case study that not only identifies the single biggest opportunity but also provides the rich context, supporting evidence, and a detailed, phased action plan to capitalize on it.

# Your mission: Transform the provided CSV data into a compelling, multi-faceted case study. You must use your tools to build a complete narrative, exploring the "what," the "why," and the "how" to drive significant business growth.

# Your Tools:
# 1. `eda_fact_sheet(df_path)`: Use once at the start for a foundational understanding.
# 2. `python_repl_ast`: This is your primary investigation tool. You are expected to use it multiple times to build a rich, interconnected story.

# ---

# ### Your Mandated Workflow:

# 1.  **Initial Recon:** Use `eda_fact_sheet` to map out the dataset.
# 2.  **Multi-Angle Investigation:** Your investigation must be thorough. Use `python_repl_ast` to explore multiple angles to build your case study. You MUST answer layered questions to get the full picture:
#     - **Overall Market Trends:** What are the year-over-year trends for `Total Volume` and `AveragePrice` for both conventional and organic types? Is the market growing?
#     - **Profitability Hotspots:** Which 5-10 specific regions have the highest *and* lowest `AveragePrice` for organic avocados? What is the exact price difference?
#     - **Supply & Demand Mismatch:** In your most profitable "hotspot" regions, what is their share of the `Total Volume`? Are we undersupplying our best markets?
#     - **Consumer Behavior:** In these top-tier regions, what are the sales trends for `Small Bags` vs. `Large Bags`? Does this tell us anything about the customer?
#     - **Seasonality:** Which months or quarters show the highest `AveragePrice` for organic avocados in these premium regions? Can we leverage this seasonality?
# 3.  **Synthesize and Report:** Weave your findings into the mandatory "Business Case Study" format below. You must connect the dots between the different data points to tell a coherent story.

# ---

# ### Mandatory Report Format: Business Case Study & Strategic Plan

# **Subject: Deep-Dive Case Study: Unlocking the Premium Organic Avocado Market**

# **(1) Executive Summary:**
# *A brief, 3-4 sentence overview. State the overall market condition, the primary opportunity you've identified, and the high-level recommendation.*
# (Example: "While our overall avocado sales are growing, we are operating with a one-size-fits-all strategy in a market that has distinct premium segments. This case study reveals a significant, untapped opportunity in specific high-end urban markets for organic avocados. Our go-forward plan outlines a phased approach to realign supply, optimize pricing, and capture this overlooked profit.")

# **(2) The Overall Market Landscape (2015-2018):**
# *Provide a brief overview of the big picture. What are the macro trends?*
# -   Present the year-over-year growth in `Total Volume` for both conventional and organic avocados.
# -   Show the trend in `AveragePrice` for both types over the four years, noting any significant spikes or dips.

# **(3) The Core Insight: The Anatomy of a Premium Market**
# *This is the heart of your case study. Detail your main finding with specific, hard numbers.*
# -   Identify the top 3-5 "Premium Regions" that show the highest `AveragePrice` for organic avocados. List the regions and their prices.
# -   Contrast this with the bottom 3-5 "Economy Regions" with the lowest `AveragePrice` for conventional avocados.
# -   State the exact dollar and percentage difference in `AveragePrice` between the top premium region and the bottom economy region.
# -   Crucially, expose the supply/demand mismatch: state the percentage of `Total Volume` these top premium regions currently receive versus their price potential.

# **(4) Supporting Analysis: Deeper Insights into the Premium Customer**
# *Add layers to your story with more detailed findings.*
# -   **Seasonal Price Peaks:** Identify the specific months or quarters when the `AveragePrice` in your premium regions is at its highest.
# -   **Purchasing Habits:** Analyze the ratio of `Small Bags` to `Large Bags` sold in the premium regions. Contrast this with the ratio in economy regions to make an inference about the customer (e.g., "Premium customers prefer smaller, more frequent purchases").

# **(5) The Go-Forward Strategic Plan: A Phased Approach**
# *Provide a detailed, multi-step action plan. This must be more than just a list; it should be a strategy.*

# **Phase 1: Immediate Realignment & Profit Capture (Next 30 Days)**
# 1.  **Supply Chain Pivot:** Detail the exact percentage of organic supply to be rerouted from specific low-margin regions to the identified high-margin "Premium Regions."
# 2.  **Initial Price Adjustment:** Specify the initial percentage price increase for organic avocados in the "Premium Regions" to test the market's price elasticity.

# **Phase 2: Growth & Market Domination (Next Quarter)**
# 1.  **Seasonal Strategy:** Launch a marketing campaign in the "Premium Regions" just before the identified seasonal price peak, emphasizing scarcity and quality to justify premium pricing.
# 2.  **Product Mix Optimization:** Based on the bag size analysis, adjust the inventory mix in premium vs. economy regions to match local consumer purchasing habits.

# **Metrics for Success:**
# -   We will measure the success of this plan by tracking the following KPIs over the next quarter:
#     -   Increase in average profit margin per unit in "Premium Regions."
#     -   Overall revenue growth from the organic category.
#     -   Change in sales volume in "Premium Regions" following the price adjustment.

# ---

# ### **CRITICAL DIRECTIVES:**
# - **DEPTH IS MANDATORY:** A shallow report is a failed report. You must use `python_repl_ast` to find specific regions, numbers, percentages, and seasonal trends.
# - **CONNECT THE DOTS:** Do not just list facts. Explain how regional price differences, supply volumes, and consumer bag preferences all tell the same story.
# - **STRATEGIC, NOT JUST ACTIONABLE:** Your plan should have phases and clear metrics for success, showing a thoughtful, long-term approach."""

In [39]:
df_path = r"D:\Code Assistant\avocado.csv"

task_prompt = (
        f"Find the instructions given by the user here : {Instructions} and follow this {Reporter_prompt} to the letter."
    )

In [40]:
from langchain.agents import AgentExecutor, create_tool_calling_agent

agent_executor = AgentExecutor(
            agent=Analyzer_agent,
            tools=tools,
            verbose=True,
            handle_parsing_errors=True,
            return_intermediate_steps=True
        )

In [41]:
result = agent_executor.invoke({
                "input": task_prompt,
                "df_path": df_path,
                "chat_history": []     
            })



> Entering new AgentExecutor chain...

Invoking: `eda_fact_sheet` with `{'path': 'D:\\Code Assistant\\avocado.csv'}`


{'n_rows': 18249, 'n_columns': 13, 'batches': [{'columns': {'Date': {'dtype': 'object', 'null_percent': 0.0, 'unique_percent': 0.93, 'examples': ['2018-02-11', '2015-11-01', '2016-01-24'], 'top_values': {'2015-12-27': 108, '2015-12-20': 108, '2015-12-13': 108}, 'avg_length': 10.0, 'max_length': 10, 'unusual_char_percent': 100.0}, 'AveragePrice': {'dtype': 'float64', 'null_percent': 0.0, 'unique_percent': 1.42, 'examples': [0.82, 0.97, 1.44], 'top_values': {'1.15': 202, '1.18': 199, '1.08': 194}, 'min': 0.44, 'max': 3.25, 'mean': 1.405978409775878, 'std': 0.40267655549555065, 'has_outliers': True}, 'Total Volume': {'dtype': 'float64', 'null_percent': 0.0, 'unique_percent': 99.93, 'examples': [218329.7, 391514.43, 3907.4], 'top_values': {'569349.05': 2, '13234.04': 2, '2858.31': 2}, 'min': 84.56, 'max': 62505646.52, 'mean': 850644.013008932, 'std': 3453545.3553994712, 